# Douyin Live Recorder

Notebook นี้ใช้สำหรับรัน DouyinLiveRecorder เพื่อบันทึกวิดีโอไลฟ์จาก Douyin (TikTok จีน)

In [1]:
# Clone repositories
!git clone https://github.com/ihmily/DouyinLiveRecorder.git
!git clone https://github.com/chaiz64/l777k.git

Cloning into 'DouyinLiveRecorder'...
remote: Enumerating objects: 1420, done.
remote: Counting objects: 100% (689/689), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 1420 (delta 553), reused 521 (delta 515), pack-reused 731 (from 3)
Receiving objects: 100% (1420/1420), 18.17 MiB | 12.56 MiB/s, done.
Resolving deltas: 100% (892/892), done.
Cloning into 'l777k'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 103 (delta 61), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 313.37 KiB | 2.00 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [2]:
# Copy configuration files
!cp l777k/config.ini DouyinLiveRecorder/config/
!cp l777k/URL_config.ini DouyinLiveRecorder/config/

In [3]:
# เข้าไปยังโฟลเดอร์โปรเจค
%cd DouyinLiveRecorder

# ติดตั้ง dependencies
!pip3 install -r requirements.txt

/content/DouyinLiveRecorder
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14576 sha256=b344312dfaa2fad2d01425d40fc626b51f1e8af1ebf6f108272d13ce4db9f1b7
  Stored in directory: /root/.cache/pip/wheels/0e/29/b2/02d071c92586aa5dbfd67cda8aea1a680900a530217d5ecf6c
Successfully built PyExecJS


In [4]:
# ติดตั้ง ffmpeg
!apt update
!apt install ffmpeg -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,721 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,994 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,246 kB]
Get:13 http://archive.ubuntu.com/u

In [5]:
# รันโปรแกรม
!python main.py

序号617 小阿鱼鱼 等待直播... 
序号103 小苏可 等待直播... 
序号100 童蕾蕾 等待直播... 
序号618 茜茜baby👑 等待直播... 
序号619 妍baby 等待直播... 
序号113 人间小奶猫 等待直播... 
序号620 YY姐 等待直播... 
序号622 楠玉小怪兽 等待直播... 
序号621 荔枝白兰地 等待直播... 
序号124 键盘月牙儿 等待直播... 
序号93 蹦迪的啾咪 等待直播... 
共监测1647个直播中 | 同一时间访问网络的线程数: 3 | 是否开启代理录制: 是 | 录制分段开启: 否 | 录制视频质量为: 原画 | 录制视频格式为: MP4 | 目前瞬时错误数为: 0 | 当前时间: 05:19:09
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
正在录制32个直播: 
序号577 江一玥(5_6🎂)[原画] 正在录制中 0:00:11
序号389 宝宝168(徐玉梅)[原画] 正在录制中 0:00:55
序号174 雨子酱Yuu[原画] 正在录制中 0:01:45
序号276 方忆沐[原画] 正在录制中 0:01:21
序号425 杨清也🥕[原画] 正在录制中 0:00:46
序号558 微小包包子[原画] 正在录制中 0:00:15
序号230 我不妮姐🍯[原画] 正在录制中 0:01:31
序号408 荆兮兮[原画] 正在录制中 0:00:50
序号509 静静不静-[原画] 正在录制中 0:00:27
序号442 芊雨_⁹²⁷⁷✨[原画] 正在录制中 0:00:42
序号554 娜娜子🎹[原画] 正在录制中 0:00:16
序号570 舞蹈生诗诗⁵²⁴⁴[原画] 正在录制中 0:00:12
序号417 美欧尼[原画] 正在录制中 0:00:48
序号401 大美的de💃中国舞[原画] 正在录制中 0:00:52
序号526 刘倩妤(潜水版[原画] 正在录制中 0:00:23
序号50 星呆露^_🐰[原画] 正在录制中 0:02:14
序号331 夏小晚🎤[原画] 正在录制中 0:01:08
序号98 腼腆小美(5_7🎂)[原画] 正在录制中 0:02:02
序号196 林恩(5月4日晚8点生日庆)[原画] 正在录

In [ ]:
import shutil
from google.colab import files

# กำหนดชื่อไฟล์ ZIP
zip_filename = "/content/douyin_live_backup.zip"
source_folder = "/content/DouyinLiveRecorder/downloads/抖音直播"

# บีบอัดโฟลเดอร์เป็นไฟล์ ZIP
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', source_folder)

# ดาวน์โหลดไฟล์ ZIP
files.download(zip_filename)


In [6]:
import os
import IPython.display as display

# ฟังก์ชันแปลงขนาดไฟล์เป็น GB หรือ TB
def convert_size(size_in_bytes):
    if size_in_bytes < 1024:
        return f"{size_in_bytes} B"
    elif size_in_bytes < 1024**2:
        return f"{size_in_bytes / 1024:.2f} KB"
    elif size_in_bytes < 1024**3:
        return f"{size_in_bytes / 1024**2:.2f} MB"
    elif size_in_bytes < 1024**4:
        return f"{size_in_bytes / 1024**3:.2f} GB"
    else:
        return f"{size_in_bytes / 1024**4:.2f} TB"

# กำหนดโฟลเดอร์หลักที่เก็บไฟล์
base_folder = "/content/DouyinLiveRecorder/downloads/抖音直播"

# ตรวจสอบว่าโฟลเดอร์มีอยู่หรือไม่
if not os.path.exists(base_folder):
    print("❌ ไม่พบโฟลเดอร์ที่กำหนด")
else:
    # ค้นหาไฟล์ทั้งหมดในโฟลเดอร์และโฟลเดอร์ย่อย
    file_paths = []
    for root, _, files in os.walk(base_folder):
        for file in files:
            if file.endswith(('.mp4', '.ts')):  # เลือกเฉพาะไฟล์ .mp4 และ .ts
                full_path = os.path.join(root, file)
                relative_path = full_path.replace(base_folder + "/", "")  # แสดงพาธย่อ
                file_paths.append((full_path, relative_path))

    # ตรวจสอบว่าพบไฟล์หรือไม่
    if not file_paths:
        print("❌ ไม่พบไฟล์ .mp4 หรือ .ts ในโฟลเดอร์")
    else:
        print(f"✅ พบ {len(file_paths)} ไฟล์ในโฟลเดอร์ '{base_folder}'\n")

        # แสดงลิงก์ดาวน์โหลดและขนาดไฟล์
        for full_path, relative_path in sorted(file_paths):
            file_size = os.path.getsize(full_path)
            file_size_str = convert_size(file_size)  # แปลงขนาดไฟล์เป็นหน่วยที่เหมาะสม
            display.display(display.HTML(
                f'<a href="/files{full_path}" download="{relative_path}" target="_blank">📥 ดาวน์โหลด {relative_path} ({file_size_str})</a><br>'
            ))


✅ พบ 32 ไฟล์ในโฟลเดอร์ '/content/DouyinLiveRecorder/downloads/抖音直播'



In [18]:
import os
from google.colab import files
import time

# --- Configuration ---
# Define configuration settings for the download process.
config = {
    'base_download_directory': '/content/DouyinLiveRecorder/downloads/抖音直播/',
    'DOWNLOAD_LIMIT': 1, # Maximum number of files to download in each batch (now 1 file per step for continuous download).
    'PROGRESS_FILE': '/content/download_progress.txt' # File to save download progress.
}

def get_last_downloaded_index(progress_file):
    """
    Reads the index of the last downloaded file from the progress file.
    Returns 0 if the file doesn't exist or is unreadable, to start from the beginning.
    """
    try:
        with open(progress_file, 'r') as f:
            content = f.read().strip()
            if content.isdigit():
                return int(content)
            return 0 # If content is not a digit, start from 0.
    except FileNotFoundError:
        return 0 # If file not found, start from 0.
    except Exception as e:
        print(f"Warning: Could not read progress file '{progress_file}'. Starting from beginning. Error: {e}")
        return 0

def update_last_downloaded_index(progress_file, index):
    """
    Saves the index of the next file to be downloaded to the progress file.
    """
    try:
        with open(progress_file, 'w') as f:
            f.write(str(index))
        print(f"Download progress saved: Next file index is {index}.")
    except Exception as e:
        print(f"Error: Could not save download progress to '{progress_file}'. Error: {e}")

def scan_and_download_files(config):
    """
    Scans the specified directory and its subdirectories for files.
    Then, downloads them one by one to the local machine, limited by batch size.
    Saves download progress to a file for continuous operation.

    Args:
        config (dict): A dictionary containing configuration parameters, including
                       'base_download_directory', 'DOWNLOAD_LIMIT', and 'PROGRESS_FILE'.
    """
    base_directory = config.get('base_download_directory')
    download_limit = config.get('DOWNLOAD_LIMIT', 1) # Ensure default is 1 for this continuous mode
    progress_file = config.get('PROGRESS_FILE', '/content/download_progress.txt')

    if not base_directory:
        print("Error: 'base_download_directory' is not specified in the configuration.")
        return

    if not os.path.isdir(base_directory):
        print(f"Error: Directory '{base_directory}' not found.")
        print("Please ensure the path is correct and the directory exists in your Colab environment.")
        return

    print(f"Scanning for files in: {base_directory}")
    found_files = []

    # Use os.walk to traverse directories and subdirectories.
    # Use sorted(filenames) to ensure consistent file order across runs.
    for root, _, filenames in os.walk(base_directory):
        for filename in sorted(filenames):
            full_path = os.path.join(root, filename)
            found_files.append(full_path)

    if not found_files:
        print(f"No files found in '{base_directory}' or its subdirectories.")
        return

    # --- Initial Prompt (only once at the beginning) ---
    start_index = get_last_downloaded_index(progress_file)

    if start_index >= len(found_files):
        print("\nAll available files have been downloaded!")
        print(f"The progress file '{progress_file}' indicates completion. To restart, you can manually delete this file in your Colab files.")
        return # Exit if all files are already downloaded

    print(f"\nFound {len(found_files)} file(s) in total (starting from index {start_index}):")
    # Display all files with their status.
    for i, f in enumerate(found_files):
        relative_path = os.path.relpath(f, base_directory)
        prefix = ""
        if i < start_index:
            prefix = "  [DOWNLOADED] " # Files already downloaded.
        elif i >= start_index and i < start_index + download_limit:
            prefix = "  [TO DOWNLOAD] " # Files to be downloaded in this batch.
        else:
            prefix = "  [QUEUED] " # Files queued for subsequent batches.
        print(f"{prefix}{i+1}. {relative_path}")

    # Prompt for initial confirmation before starting the continuous download.
    user_input = input("\nDo you want to start downloading files? (yes/no/reset): ").lower().strip()
    if user_input == 'reset':
        update_last_downloaded_index(progress_file, 0)
        print("Download progress reset. Please run the cell again to start from the beginning.")
        return # Exit the function to allow a fresh start.
    elif user_input != 'yes':
        print("Download cancelled by user.")
        return # Exit if cancelled.

    print("\nStarting continuous download process...")

    # --- Continuous Download Loop ---
    while True:
        current_start_index = get_last_downloaded_index(progress_file)

        if current_start_index >= len(found_files):
            print("\nAll available files have been downloaded!")
            print(f"The progress file '{progress_file}' indicates completion. To restart, you can manually delete this file in your Colab files.")
            break # Exit the loop if all files are downloaded

        # --- Select files for the current batch (always 1 file now) ---
        files_to_download_current_batch = found_files[current_start_index : current_start_index + download_limit]

        if not files_to_download_current_batch:
            print("No files left to download in this batch based on the current progress and limit.")
            print(f"Current start index: {current_start_index}. Total files: {len(found_files)}. Download limit: {download_limit}.")
            break # Exit if no files are selected for the current batch.

        files_downloaded_in_batch = 0
        for i, file_path in enumerate(files_to_download_current_batch):
            try:
                relative_path = os.path.relpath(file_path, base_directory)
                # Displaying actual index for clarity in continuous mode
                actual_file_number = current_start_index + i + 1
                print(f"[{actual_file_number}/{len(found_files)}] Downloading: {relative_path}")

                files.download(file_path)
                print(f"  --> Download initiated for '{relative_path}'.")
                files_downloaded_in_batch += 1

                # Add a longer delay to prevent issues with rapid consecutive downloads.
                # คุณสามารถปรับค่า 5 ให้เป็นค่าที่เหมาะสมกับเบราว์เซอร์ของคุณ เช่น 5, 10, 15 วินาที
                time.sleep(10) # <--- เปลี่ยนค่าตรงนี้!

            except FileNotFoundError:
                print(f"  Error: File not found at '{file_path}'. Skipping.")
            except Exception as e:
                print(f"  An unexpected error occurred during download of '{file_path}': {e}. Skipping.")

        # --- Update progress after the current batch is completed ---
        next_start_index = current_start_index + files_downloaded_in_batch
        update_last_downloaded_index(progress_file, next_start_index)

        # No prompt here, the loop will automatically continue to the next file
        # if there are more files remaining.
        if next_start_index < len(found_files):
            print(f"\nProceeding to the next file...")
        else:
            print("\nAll available files have been downloaded!")
            print(f"The progress file '{progress_file}' indicates completion. To restart, you can manually delete this file.")
            break # Exit the loop if all files are downloaded.

# --- Run the function ---
scan_and_download_files(config)


Scanning for files in: /content/DouyinLiveRecorder/downloads/抖音直播/

Found 32 file(s) in total (starting from index 0):
  [TO DOWNLOAD] 1. 荆兮兮/荆兮兮_2025-06-01_05-18-18.mp4
  [QUEUED] 2. Theone🕊/Theone🕊_2025-06-01_05-18-56.mp4
  [QUEUED] 3. 夏语曦🎋/夏语曦🎋_2025-06-01_05-16-58.mp4
  [QUEUED] 4. 桃盈盈🍧/桃盈盈🍧_2025-06-01_05-17-43.mp4
  [QUEUED] 5. 廖廖/廖廖_2025-06-01_05-17-16.mp4
  [QUEUED] 6. 夏小晚🎤/夏小晚🎤_2025-06-01_05-18-00.ts
  [QUEUED] 7. 江一玥(5_6🎂)/江一玥(5_6🎂)_2025-06-01_05-18-58.ts
  [QUEUED] 8. 腼腆小美(5_7🎂)/腼腆小美(5_7🎂)_2025-06-01_05-17-06.mp4
  [QUEUED] 9. 是Lily啊/是Lily啊_2025-06-01_05-16-49.mp4
  [QUEUED] 10. 小梦🧚中国舞/小梦🧚中国舞_2025-06-01_05-18-58.ts
  [QUEUED] 11. 星呆露^_🐰/星呆露^_🐰_2025-06-01_05-16-55.mp4
  [QUEUED] 12. 高夕妍👻(五一休假啦)/高夕妍👻(五一休假啦)_2025-06-01_05-18-15.mp4
  [QUEUED] 13. 林恩(5月4日晚8点生日庆)/林恩(5月4日晚8点生日庆)_2025-06-01_05-17-30.ts
  [QUEUED] 14. 方忆沐/方忆沐_2025-06-01_05-17-47.mp4
  [QUEUED] 15. 娜娜子🎹/娜娜子🎹_2025-06-01_05-18-53.mp4
  [QUEUED] 16. 舞蹈生诗诗⁵²⁴⁴/舞蹈生诗诗⁵²⁴⁴_2025-06-01_05-18-56.mp4
  [QUEUED] 17. 微小包包子/微小包包子_20

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  --> Download initiated for '荆兮兮/荆兮兮_2025-06-01_05-18-18.mp4'.
Download progress saved: Next file index is 1.

Proceeding to the next file...
[2/32] Downloading: Theone🕊/Theone🕊_2025-06-01_05-18-56.mp4


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

  --> Download initiated for '佳怡violinist🎻/佳怡violinist🎻_2025-06-01_05-17-14.mp4'.
Download progress saved: Next file index is 31.

Proceeding to the next file...
[32/32] Downloading: 紫哥/紫哥_2025-06-01_05-17-31.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  --> Download initiated for '紫哥/紫哥_2025-06-01_05-17-31.mp4'.
Download progress saved: Next file index is 32.

All available files have been downloaded!
The progress file '/content/download_progress.txt' indicates completion. To restart, you can manually delete this file.
